In [8]:
from transformers import LEDForConditionalGeneration, AutoConfig, AutoTokenizer, BigBirdPegasusForConditionalGeneration, PegasusForConditionalGeneration
import torch
import pandas as pd
import numpy as np
import wandb
from pathlib import Path

In [3]:
dummy_input = 'Device and Usage Requirements. Certain types of Paid Services may require you to use a device that meets the system and compatibility requirements for such Paid Service, which may change from time to time.'

In [4]:
text = '''4. Your license to use the products
Blackboard gives you a personal, worldwide, royalty-free, non-assignable, and non-exclusive license to use the Products, subject to these Terms and any agreement entered into by your institution relating to the Products, solely for the intended use of the Products, and not for competitive intelligence, analysis, or demonstrations. You may not copy, modify, distribute, sell, or lease any part of our Products, nor may you reverse engineer or attempt to extract the source code of any Product, unless laws prohibit those restrictions or you have our written permission.

5. Mobile software
A. Mobile Software

We may make available software to access the Products via a mobile device (“Mobile Software”). To use the Mobile Software you must have a mobile device that is compatible with the Mobile Software. Blackboard does not warrant that the Mobile Software will be compatible with your mobile device. You may use mobile data in connection with the Mobile Software and may incur additional charges from your wireless provider for these services. You agree that you are solely responsible for any such charges. Blackboard hereby grants you a non-exclusive, non-transferable, revocable license to use a compiled code copy of the Mobile Software for one account on one mobile device owned or leased solely by you, for your personal use. You may not: (i) modify, disassemble, decompile or reverse engineer the Mobile Software, except to the extent that such restriction is expressly prohibited by law; (ii) rent, lease, loan, resell, sublicense, distribute or otherwise transfer the Mobile Software to any third party or use the Mobile Software to provide time sharing or similar services for any third party; (iii) make any copies of the Mobile Software; (iv) remove, circumvent, disable, damage or otherwise interfere with security-related features of the Mobile Software, features that prevent or restrict use or copying of any content accessible through the Mobile Software, or features that enforce limitations on use of the Mobile Software; or (v) delete the copyright and other proprietary rights notices on the Mobile Software. You acknowledge that Blackboard may from time to time issue upgraded versions of the Mobile Software, and may automatically electronically upgrade the version of the Mobile Software that you are using on your mobile device. You consent to such automatic upgrading on your mobile device, and agree that the terms and conditions of this Agreement will apply to all such upgrades. Any third-party code that may be incorporated in the Mobile Software is covered by the applicable open source or third-party license EULA, if any, authorizing use of such code. The foregoing license grant is not a sale of the Mobile Software or any copy thereof, and Blackboard or its third-party partners or suppliers retain all right, title, and interest in the Mobile Software (and any copy thereof). Any attempt by you to transfer any of the rights, duties or obligations hereunder, except as expressly provided for in this Agreement, is void. Blackboard reserves all rights not expressly granted under these Terms. The Mobile Software originates in the United States, and is subject to United States export laws and regulations. The Mobile Software may not be exported or re-exported to certain countries or those persons or entities prohibited from receiving exports from the United States. In addition, the Mobile Software may be subject to the import and export laws of other countries. You agree to comply with all United States and foreign laws related to use of the Mobile Software and the Products.

B. Mobile Software from App Store by Apple

The following applies to any Mobile Software you acquire from the App Store (“App Store-Sourced Software”): You acknowledge and agree that these Terms are solely between you and Blackboard, not Apple, and that Apple has no responsibility for the App Store-Sourced Software or content thereof. Your use of the App Store-Sourced Software must comply with the App Store Terms of Service. You acknowledge that Apple has no obligation whatsoever to furnish any maintenance and support services with respect to the App Store-Sourced Software. In the event of any failure of the App Store-Sourced Software to conform to any applicable warranty, you may notify Apple, and Apple will refund the purchase price for the App Store-Sourced Software to you; to the maximum extent permitted by applicable law, Apple will have no other warranty obligation whatsoever with respect to the App Store-Sourced Software, and any other claims, losses, liabilities, damages, costs or expenses attributable to any failure to conform to any warranty will be solely governed by these Terms and any law applicable to Blackboard as provider of the software. You acknowledge that Apple is not responsible for addressing any of your claims or those of any third party relating to the App Store-Sourced Software or your possession and/or use of the App Store-Sourced Software, including, but not limited to: (i) product liability claims; (ii) any claim that the App Store-Sourced Software fails to conform to any applicable legal or regulatory requirement; and (iii) claims arising under consumer protection or similar legislation; and all such claims are governed solely by this Agreement and any law applicable to Blackboard as provider of the software. You acknowledge that, in the event of any third-party claim that the App Store-Sourced Software or your possession and use of that App Store-Sourced Software infringes that third party’s intellectual property rights, Blackboard, not Apple, will be solely responsible for the investigation, defense, settlement and discharge of any such intellectual property infringement claim to the extent required by these Terms. You and Blackboard acknowledge and agree that Apple, and Apple’s subsidiaries, are third-party beneficiaries of these Terms as relates to your license of the App Store-Sourced Software, and that, upon your acceptance of the Terms, Apple will have the right (and will be deemed to have accepted the right) to enforce these Terms as relates to your license of the App Store-Sourced Software against you as a third-party beneficiary thereof.

C. Mobile Software from Google Play Store.

The following applies to any Mobile Software you acquire from the Google Play Store (“Google-Sourced Software”): (i) you acknowledge that the Terms are between you and Blackboard only, and not with Google, Inc. (“Google”); (ii) your use of Google-Sourced Software must comply with Google’s then-current Google Play Store Terms of Service; (iii) Google is only a provider of the Google Play Store where you obtained the Google-Sourced Software; (iv) Blackboard, and not Google, is solely responsible for its Google-Sourced Software; (v) Google has no obligation or liability to you with respect to Google-Sourced Software or the Terms; and (vi) you acknowledge and agree that Google is a third-party beneficiary to the Terms as relates to Blackboard’s Google-Sourced Software.

D. Mobile Software from Microsoft Store

The following applies to any Mobile Software you acquire from the Microsoft Store ("MS-Sourced Software"): (i) you acknowledge that the Agreement is between you and Blackboard only, and not with Microsoft Corporation ("Microsoft"); (ii) your use of MS-Sourced Software must comply with Microsoft's then-current Microsoft Store Terms of Service; (iii) Microsoft is only a provider of the Microsoft Store where you obtained the MS-Sourced Software; (iv) Blackboard, and not Microsoft, is solely responsible for its MS-Sourced Software; (v) Microsoft has no obligation or liability to you with respect to MS-Sourced Software or this Agreement; and (vi) you acknowledge and agree that Microsoft is a third-party beneficiary to the Agreement as it relates to Blackboard’s MS-Sourced Software.

6. Blackboard rights
The Products are always evolving and the form, nature, and/or functionality of the Products may change from time to time without prior notice to you. In addition, Blackboard may stop (permanently or temporarily) providing the Products (or any features within the Products) to you or to users generally and may not be able to provide you with prior notice. We also retain the right to create limits on use and storage at our sole discretion at any time without prior notice to you.

All right, title, and interest in and to the Products (excluding Content provided by users or other third parties) are and will remain the exclusive property of Blackboard and its licensors. The Products are protected by copyright, trademark, and other laws of both the United States and foreign countries. Nothing in the Terms gives you a right to use the Blackboard name or any of the Blackboard trademarks, logos, domain names, and other distinctive brand features. Any feedback, comments, or suggestions you may provide regarding Blackboard, or the Products is entirely voluntary and we will be free to use such feedback, comments or suggestions as we see fit and without any obligation to you.

7. Registration obligations & passwords
You may never use another User’s account without permission. In consideration of your use of the Products, you agree (a) that Your Information will be true, accurate, current and complete, and (b) to maintain and promptly update Your Information to keep it true, accurate, current and complete. You are responsible for safeguarding any password that you use to access the Products (including passwords used on other Blackboard products that permit single sign-on to the Products) and for any activities or actions under your password. We encourage you to use “strong” passwords (passwords that use a combination of upper and lower case letters, numbers and symbols) with your account. You agree to (a) notify Blackboard of any unauthorized use of your password or user identification and any other breach of security, and (b) ensure that you exit from your account at the end of each session.

8. Export control
As required by the laws of the United States and other countries: (a) you understand that the Products are subject to export controls under the U.S. Commerce Department's Export Administration Regulations ("EAR"); (b) you are not located in a prohibited destination country under the EAR or U.S. sanctions regulations; and (c) you will not export, re-export, or transfer the Products to any prohibited destination or persons or entities on the U.S. Bureau of Industry and Security Denied Parties List or Entity List, or the U.S. Office of Foreign Assets Control list of Specially Designated Nationals and Blocked Persons, or any similar lists maintained by other countries, without the necessary export license(s) or authorization(s).

9. Termination
We may suspend or terminate your accounts or cease providing you with all or part of the Products at any time for any reason, including, but not limited to, if we reasonably believe: (i) you have violated these Terms, (ii) you create risk or possible legal exposure for us; or (iii) our provision of the Products is no longer commercially viable. In such a termination event, the license granted hereunder shall automatically terminate. In all such cases, the Terms shall terminate, including, without limitation, your license to use the Products, except that the following Sections shall continue to apply: 2, 3, 5, 6, 9, 10, and 11.

Further, if you stop using the Products for more than 60 days, your accounts may be deleted and Blackboard assumes no responsibility for Content deleted following inactivity. If you wish to delete your Blackboard Product account(s) in their entirety(ies), you may contact us via GeneralCounsel@blackboard.com and specify which Blackboard Product account(s) you wish to delete. We will delete your specified accounts within a reasonable time, unless prohibited by law.

Nothing in this Section shall affect Blackboard’s rights to change, limit, or stop the provision of the Products without prior notice, as provided above in Section 5.

10. Disclaimers & limitations of liability
Please read this Section carefully since it limits the liability of Blackboard and its parents, subsidiaries, affiliates, related companies, officers, directors, employees, agents, representatives, partners, and/or licensors (collectively, the “Blackboard Entities”). Each of the Subsections below only applies up to the maximum extent permitted under applicable law. Nothing in this Section is intended to limit any rights you may have which may not be lawfully limited.

A. The Products are available “AS-IS”

Your access to and use of the Products or any Content are at your own risk. You understand and agree that the Products are provided to you on an “AS IS” and “AS AVAILABLE” basis. Without limiting the foregoing, to the maximum extent permitted under applicable law, THE BLACKBOARD ENTITIES DISCLAIM ALL WARRANTIES AND CONDITIONS, WHETHER EXPRESS OR IMPLIED, OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR NON-INFRINGEMENT. No advice or information, whether oral or written, obtained from any of the Blackboard Entities or through the Products, will create any warranty not expressly made herein.

B. HIPAA Disclaimer

Blackboard does not warrant or provide any assurances that your use of the Products will comply with the Health Insurance Portability and Accountability Act of 1996, as amended (“HIPAA”). If you are a health care provider, health plan, insurer, health care information clearinghouse or other entity that may be considered a “covered entity” under HIPAA or a “business associate” of a “covered entity” under HIPAA, you shall take all steps necessary to comply with HIPAA and shall be responsible for any applicable compliance with HIPAA. You agree, that you will not provide Blackboard with any protected health information (as that term is defined under HIPAA) and that Blackboard is therefore not deemed to be a business associate of Client for purposes of compliance with HIPAA.

C. Links

The Products may contain links to third-party websites or resources. You acknowledge and agree that the Blackboard Entities are not responsible or liable for: (i) the availability or accuracy of such websites or resources; or (ii) the content, products, or services on or available from such websites or resources. Links to such websites or resources do not imply any endorsement by the Blackboard Entities of such websites or resources or the content, products, or services Available from such websites or resources. You acknowledge sole responsibility for and assume all risk arising from your use of any such websites or resources.

D. Limitation of liability

TO THE MAXIMUM EXTENT PERMITTED BY APPLICABLE LAW, THE BLACKBOARD ENTITIES SHALL NOT BE LIABLE FOR ANY INDIRECT, INCIDENTAL, SPECIAL, CONSEQUENTIAL OR PUNITIVE DAMAGES, OR ANY LOSS OF PROFITS OR REVENUES, WHETHER INCURRED DIRECTLY OR INDIRECTLY, OR ANY LOSS OF DATA, USE, GOOD-WILL, OR OTHER INTANGIBLE LOSSES, RESULTING FROM (i) YOUR ACCESS TO OR USE OF OR INABILITY TO ACCESS OR USE THE PRODUCTS; (ii) ANY CONDUCT OR CONTENT OF ANY THIRD PARTY ON,THROUGH, OR ASSOCIATED WITH THE PRODUCTS, INCLUDING WITHOUT LIMITATION, ANY DEFAMATORY, OFFENSIVE OR ILLEGAL CONDUCT OF OTHER USERS OR THIRD PARTIES; (iii) ANY CONTENT OBTAINED FROM THE PRODUCTS; OR (iv) UNAUTHORIZED ACCESS, USE OR ALTERATION OF YOUR TRANSMISSIONS OR CONTENT.

IN NO EVENT SHALL THE AGGREGATE LIABILITY OF THE BLACKBOARD ENTITIES EXCEED THE GREATER OF ONE HUNDRED U.S. DOLLARS (U.S. $100.00) OR THE AMOUNT YOU PAID BLACKBOARD, IF ANY, IN THE PAST SIX MONTHS FOR THE PRODUCTS GIVING RISE TO THE CLAIM.

THE LIMITATIONS OF THIS SUBSECTION SHALL APPLY TO ANY THEORY OF LIABILITY, WHETHER BASED ON WARRANTY, CONTRACT, STATUTE, TORT (INCLUDING NEGLIGENCE) OR OTHERWISE, AND WHETHER OR NOT THE BLACKBOARD ENTITIES HAVE BEEN INFORMED OF THE POSSIBILITY OF ANY SUCH DAMAGE, AND EVEN IF A REMEDY SET FORTH HEREIN IS FOUND TO HAVE FAILED OF ITS ESSENTIAL PURPOSE.

11. General terms
A. Waiver & severability

The failure of Blackboard to enforce any right or provision of these Terms will not be deemed a waiver of such right or provision. In the event that any provision of these Terms is held to be invalid or unenforceable, then that provision will be limited or eliminated to the minimum extent necessary, and the remaining provisions of these Terms will remain in full force and effect.

B. Governing law

These Terms and any action related thereto will be governed by the laws of the State of New York without regard to or application of its conflict of law provisions or your state or country of residence. All claims, legal proceedings, or litigation arising in connection with the Products will be brought solely in the federal or state courts located in Washington, D.C., United States, and you consent to the jurisdiction of and venue in such courts and waive any objection as to inconvenient forum.

C. U.S. Government users & U.S. Government restricted rights

Certain of the components that comprise the Products are “commercial items” as that term is defined at 48 C.F.R. 2.101, consisting of “commercial computer software” and/or “commercial computer software documentation” as such terms are used in 48 C.F.R. 12.212. Consistent with 48 C.F.R. 12.212 and 48 C.F.R. 227.7202-1 through 227.7202-4, all U.S. Government end users acquire the Software with only those rights set forth herein.

The Products, any Mobile Software and all upgrades to the foregoing (if any) are provided with restricted rights. Use, duplication, or disclosure by the U.S. Government is subject to restrictions as set forth in subparagraph (c)(1)(ii) of the Rights in Technical Data and Computer Software clause at DFARS 252.227-7013 or subparagraphs (c)(1) and (2) of the Commercial Computer Software-Restricted Rights at 48 CFR 52.227-19, as applicable.

Contractor/manufacturer is Blackboard Inc., 1111 19th Street N.W., Washington, D.C. 20006. All rights not specifically granted in this Agreement are reserved by Blackboard.

D. Entire agreement

These Terms and our Privacy Policy are the entire and exclusive agreement between Blackboard and you regarding the Products, and these Terms supersede and replace any prior agreements between Blackboard and you regarding the Products. Other than members of the group of companies of which Blackboard Inc. is the parent, no other person or company will be third party beneficiaries to the Terms.

Notwithstanding the foregoing, nothing in these Terms supersedes or limits your rights under (1) the terms and conditions of any written agreement you have entered into with Blackboard regarding the use of Products, or (2) applicable laws or regulations to the extent these Terms are prohibited by such laws or regulations. In the event of any conflict between these Terms and the terms and conditions of an applicable written agreement you have entered into with Blackboard, the terms and conditions of the written agreement shall control.

We may revise these Terms from time-to-time, the most current version will always be at the referenced link. If the revision, in our sole discretion, is material we will notify you through the Products or through the email associated with your profile. By continuing to access or use the Products after those revisions become effective, you agree to be bound by the revised Terms.

These Products are operated and provided by Blackboard:

'''

In [5]:
def measure_model_performance(model, tokenizer):
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions=10
    timings = np.zeros((repetitions, 1))

    # GPU-WARM-UP
    for _ in range(5):
        _ = summarize(dummy_input, tokenizer)

    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):

            starter.record()
#             _ = df["original_text"].apply(lambda x: summarize(x))
            _ = summarize(text, tokenizer)
            ender.record()

            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time

    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    
    return mean_syn, std_syn

In [15]:
def summarize(text, tokenizer):
    input_tokenized = tokenizer.encode(text, return_tensors='pt', max_length=8192, truncation=True).to(device)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=200,
                                  max_length=400,
                                  early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

    return summary

In [9]:
run = wandb.init(project = "major-one", name = "Enter name")
artifact = run.use_artifact("model-longformer:v0")
datadir = artifact.download()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact model-longformer:v0, 620.89MB. 9 files... 
wandb:   9 of 9 files downloaded.  
Done. 0:0:11.7


In [10]:
config_path = datadir + "/config.json"
model_path = datadir + "/pytorch_model.bin"
vocab_path = datadir + "/vocab.json"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
config = AutoConfig.from_pretrained(config_path)
model = LEDForConditionalGeneration(config)
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [12]:
longformer_tokenizer = AutoTokenizer.from_pretrained('allenai/led-base-16384')

In [13]:
model.to(device)

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0-5): 6 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
     

In [16]:
mean_time, std_time = measure_model_performance(model, longformer_tokenizer)
print(f"Mean Inference Time: {mean_time/1000} s, Std Dev: {std_time/1000}")

Mean Inference Time: 3.290881787109375 s, Std Dev: 0.06163962314471327
